# Python Cheatsheets - EFFICIENT CODING

While using Python, it is essential to code efficiently by;
- Writing clean, readable, fast, and efficient Python code
- Profiling the code for bottlenecks
- Eliminating bottlenecks and bad design patterns
The code should run with minimum possible completion time and with minimum possible resource consumption. The Zen of Pyrthon by Tim Peters is as below:

In [1]:
import timeit
import this
import numpy

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---
### 1) Timing and Profiling

__Timing:__ When working in the IPython interactive shell or in a Jupyter notebook with a Python kernel, execution time of a Python statement or expression can be measured using the timeit module with magic command . It can be used both as a line and cell magic:

```python
%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
```
In line mode, a single-line statement is timed using the magic command with a single ```%``` mark. It is also possible to time multiple lines by using semicolons to chain expressions.

```python
%%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
```
In cell mode, the statement on the same line with the command will be executed as a setup and only be run once. It is not be timed, and the body of the cell is timed. The magic command is called with a double ```%``` mark. The cell body has access to any variables created in the setup code.

More details on the magic command can be found on the IPython docs [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit). For command-line interface and Python interface, the library [here](https://docs.python.org/3/library/timeit.html) provides other methods than magic commands.

In [5]:
# in line mode
%timeit x = [*range(5)]

344 ns ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%%timeit
# in cell mode
x = []
for i in range(5):
    x.append(i)

704 ns ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


__Code Profiling:__ Detailed stats on frequency and duration of function calls can be obtained by code profiling. It allows to analyze the code and spot any bottlenecks. ```line_profiler``` package allows to perform line-by-line analysis and can be downloaded with using Python packaging tool ```pip```. Later, the IPython extension can be loaded by:
```python
%load_ext line_profiler
```

In [17]:
%load_ext line_profiler

In [23]:
def sum_of_list(x):
    sum = 0
    for i in x:
        sum += i
    return sum

%lprun -f sum_of_list sum_of_list(range(5000))

Timer unit: 1e-07 s

Total time: 0.0048406 s
File: C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3224\3293575127.py
Function: sum_of_list at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sum_of_list(x):
     2         1          8.0      8.0      0.0      sum = 0
     3      5001      22675.0      4.5     46.8      for i in x:
     4      5000      25717.0      5.1     53.1          sum += i
     5         1          6.0      6.0      0.0      return sum

__Memory Profiling:__ 

The amount of memory an operation uses can be evaluated with memory profiling. ```memory_profiler``` package contains two useful magic functions: the ```%memit``` (which offers a memory-measuring equivalent of ```%timeit```) and the ```%mprun``` magic functions (which offers a memory-measuring equivalent of ```%lprun```). It can be downloaded with using Python packaging tool ```pip``` and the IPython extension can be loaded by:
```python
%load_ext memory_profiler
```

In [26]:
%load_ext memory_profiler

In [27]:
def sum_of_list(x):
    sum = 0
    for i in x:
        sum += i
    return sum

%memit sum_of_list(range(5000))

peak memory: 145.53 MiB, increment: 0.34 MiB


The ```%memit``` magic function is used rather simple as within the same notebook, using the function name. It returns the peak memory usage. For a line-by-line description of memory use, ```%mprun``` magic is used. However, it works only for functions defined in separate modules rather than the notebook itself. Hence, it is efficient to use ```%%file``` magic to create a simple module including the function to be evaluated, and then importing the function and using the memory profiler. 

In [32]:
%%file "module/list_ops.py" # create the module folder beforehand
def sum_of_list(x):
    sum = 0
    for i in x:
        sum += i
    return sum

Writing module/list_ops.py


In [36]:
from module.list_ops import sum_of_list
%mprun -f sum_of_list sum_of_list(range(5000))

Filename: d:\Master\Github\Python-Cheatsheets\module\list_ops.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1    158.8 MiB    158.8 MiB           1   def sum_of_list(x):
     2    158.8 MiB      0.0 MiB           1       sum = 0
     3    158.8 MiB      0.0 MiB        5001       for i in x:
     4    158.8 MiB      0.0 MiB        5000           sum += i
     5    158.8 MiB      0.0 MiB           1       return sum

Here the Increment column shows how much each line affects the total memory budget.

---
### 2) Building with Built-ins

Python provides various built-ins which should be preferred while performing basic operations.
- Built-in types: ```list```, ```tuple```, ```set```, ```dict```, etc.
- Built-in functions: ```print()```, ```len()```, ```range()```, ```round()```, ```enumerate()```, ```map()```, ```zip()```, etc.
- Built-in modules: ```os```, ```sys```, ```itertools```, ```collections```, ```math```, etc.

__Literal syntax:__ Python allows to create data structures using either a formal name (i.e. ```dict()```) or a literal syntax (i.e. ```{}```). Using a literal syntax for creating a data structure can become faster.

In [13]:
# a tuple created using the formal name
print("Creating with formal name:")
%timeit formal_tuple = tuple()

# a tuple created using the literal syntax
print("Creating with literal syntax:")
%timeit literal_tuple = ()

Creating with formal name:
60.5 ns ± 2.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
Creating with literal syntax:
27.9 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


The output of the magic command can be saved into a variable with -o flag

In [15]:
# a list created by unpacking range
times = %timeit -o nums_range = [*range(1,10)]

print("The best time:", times.best, "/ The worst time:", times.worst)

345 ns ± 6.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
The best time: 3.297941000200808e-07 / The worst time: 3.516030999599025e-07


__```range()``` function:__ For large sequence of numbers, using range is more practical.

In [39]:
# explicitly creating a list
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# creating a list by unpacking a range object
nums_range = [*range(1,10)]

# check if the results are same
if(nums == nums_range):
    print("Lists are the same:", nums_range)

Lists are the same: [1, 2, 3, 4, 5, 6, 7, 8, 9]


__```enumerate()``` function:__ For obtaining an indexed list, enumerate is more efficient. Four different version of creating an indexed list is shown below for a given list.

In [2]:
# generate a list to be indexed
chapters = ['Introduction', 'Related Work', 'Methodology', 'Results', 'Discussion', 'Conclusion']

# basic for loop
indexed_chapters = []
print("Basic loop is running:")
%timeit for i in range(len(chapters)): indexed_chapters.append((i, chapters[i]))

# loop with enumerate
indexed_chapters_enum = []
print("Enumerate loop is running:")
%timeit for i, chapter in enumerate(chapters): indexed_chapters_enum.append((i,  chapter))

# loop with enumerate using list comprehension
indexed_chapters_comp = []
print("Enumerate list comprehension is running:")
%timeit indexed_chapters_comp = [(i, chapter) for i, chapter in enumerate(chapters)]

# unpacking enumerate object with a starting index of one (FASTEST)
indexed_chapters_unpack = []
print("Enumerate unpacking is running:")
%timeit indexed_chapters_unpack = [*enumerate(chapters, 1)]

Basic loop is running:
1.46 µs ± 84.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Enumerate loop is running:
1.24 µs ± 24.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Enumerate list comprehension is running:
936 ns ± 36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Enumerate unpacking is running:
401 ns ± 6.25 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


__```map()``` function:__ To apply a function to every element of an object, map is more efficient. Three different version of applying a function to an object's elements are shown below.

In [3]:
# generate a list to be changed
chapters = ['Introduction', 'Related Work', 'Methodology', 'Results', 'Discussion', 'Conclusion']

# basic for loop
upper_chapters = []
print("Basic loop is running:")
%timeit for chapter in chapters: upper_chapters.append(chapter.upper())

# list with map
print("Map is running:")
%timeit upper_chapters_map = list(map(str.upper, chapters))

# unpacking map (FASTEST)
print("Unpacking map is running:")
%timeit upper_chapters_unpack = [*map(str.upper, chapters)]

Basic loop is running:
1.28 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Map is running:
797 ns ± 9.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Unpacking map is running:
762 ns ± 8.65 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


__```NumPy``` library:__ It is a library for working with arrays and mathematical operations in Python, which provides support for large, multi-dimensional arrays and matrices. It has the __*broadcasting*__ ability which vectorizes operations, hence all operations are performed on all elements of an object at once. It is much more efficient than using lists.

In [4]:
# generate a list to be changed
odds = [-5, -3, -1, 1, 3, 5]

# basic for loop
sqrd_odds = []
print("Basic loop is running:")
%timeit for odd in odds: sqrd_odds.append(odd ** 2)

# list with map
def sqr(i):
    return i ** 2
print("Map is running:")
%timeit sqrd_odds_map = list(map(sqr, odds))

# list comprehension 
print("List comprehension is running:")
%timeit sqrd_odds_comp  = [odd ** 2 for odd in odds]

# numpy (FASTEST)
print("NumPy is running:")
%timeit sqrd_odds_np = np.array(odds) ** 2

Basic loop is running:
2.46 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Map is running:
2.5 µs ± 60.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
List comprehension is running:
2.07 µs ± 32.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
NumPy is running:
1.54 µs ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


__```zip()``` function:__ It takes zero or more iterables (user-defined variables or built-in iterables like list, string, dict), aggregates them in a tuple, and returns it. When no parameter is passed, it returns an empty iterator. When a single parameter is passed, it returns an iterator of tuples with each having only one element, while multiple iterables as poarameters returns an iterator of tuples with each tuple having elements from all the iterables.

In [43]:
# generate lists to be zipped
numeric = [1, 2, 3, 4, 5, 6]
str_en = ['One', 'Two', 'Three', 'Four', 'Five', 'Six']
str_de = ['Eins', 'Zwei', 'Drei', 'Vier', 'Fünf', 'Sechs']
str_es = ['Uno', 'Dos', 'Tres', 'Cuatro', 'Cinco', 'Seis']

# combine string and number lists
num_en = [*zip(numeric, str_en)]
print("English numbers:", *num_en) # * is used to extract the list elements from num_en

# combine string lists in all languages
str_full = [*zip(str_en[:4], str_de[:4], str_es[:4])]
print("All numbers:", *str_full)

# combine five items from numeric and three items from spanish
num_es = [*zip(numeric[:5], str_es[:3])]
print("Spanish numbers:", *num_es)

English numbers: (1, 'One') (2, 'Two') (3, 'Three') (4, 'Four') (5, 'Five') (6, 'Six')
All numbers: ('One', 'Eins', 'Uno') ('Two', 'Zwei', 'Dos') ('Three', 'Drei', 'Tres') ('Four', 'Vier', 'Cuatro')
Spanish numbers: (1, 'Uno') (2, 'Dos') (3, 'Tres')
